**Library**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import  datetime
import calendar
import time 
from ipyparallel import Client
import zmq 



In [ ]:
#create a client
rc = Client(context=zmq.Context()) 
rc.ids

In [ ]:

dview=rc[:]

with dview.sync_imports():
    import pandas
    import matplotlib
    import matplotlib.pyplot
    import numpy 
    import  datetime
    import calendar
    import time 

In [ ]:
runtime=time.time()
def para_read(file):
    dview = rc[:]
    # PARTITION THE IMPORT BY SCATTER() #
    dview.scatter("df", pd.read_csv(file))
    return pd.concat([i for i in dview["df"]])

t2= time.time()
df= para_read('parking-citations.csv')
t2=time.time()-t2
print ('Number of data:',df.shape, "time:",t2)


In [ ]:
@dview.parallel(block = True)
def getLocation(location):

  i=0
  list= np.empty((len(location),1),dtype="U20")
  for data in location:
    temp=data.split(None, 1)[0]
    if temp.isnumeric():
      data=data[len(temp)+1:]
    list[i]=data
    i=i+1
  return list

In [ ]:

temp=  df['Location'].dropna()
location=pd.DataFrame()
location['Quantity']=np.ones(len(temp),dtype=int)
location['Location']=getLocation(temp)
location['Quantity']=location['Quantity'].groupby(location['Location']).transform('sum')
location['%'] = (location['Quantity']/len(location))*100
location=location.drop_duplicates(['Location']).sort_values(by = ['Quantity'], ascending = [False])
print(location.head(20)) 

In [ ]:
location.groupby(['Location']).sum().sort_values(by= 'Quantity', ascending=True)[-20:].plot(kind='barh',width=1.9,figsize=(14, 12),color='b')
plt.xlabel('Top 20 Location')
plt.title("Parking Ciatation - Location")
plt.show()

In [ ]:
@dview.parallel(block = True)
def create_issue_time_category(issue_time):
  issue_cat= np.zeros(24,dtype=int)
  for issue in issue_time:
    key= int(issue/100)
    issue_cat[key]=issue_cat[key]+1
  return issue_cat


issue_time= df['Issue time'].dropna()

list_issue_time=create_issue_time_category(issue_time)

issue= pd.DataFrame()
issue['Issue Time']= np.arange(24)
issue['Quantity']=list_issue_time
issue['%']=np.zeros(len(issue))
issue['%']= (issue['Quantity']/len(issue_time))*100
issue=issue.sort_values(by = ['Quantity'], ascending = [False])
print (issue.head(issue.shape[0]))


In [ ]:
@dview.parallel(block = True)
def create_list_near_issue_time(issue_time):  
  
  issue_cat= np.zeros(24,dtype=int)

  for issue in issue_time:
    key= int(issue/100)
   
    if issue%100 <30:
      issue_cat[key]=issue_cat[key]+1
    else:
      if key<23: 
        issue_cat[key+1]=issue_cat[key+1]+1
      else: 
        issue_cat[0]=issue_cat[0]+1
  return issue_cat


issue_time= df['Issue time'].dropna()

list_issue_time=create_list_near_issue_time(issue_time)

issue= pd.DataFrame()
issue['Issue Time']= np.arange(24)
issue['Quantity']=list_issue_time
issue['%']=np.zeros(len(issue))
issue['%']= (issue['Quantity']/len(issue_time))*100

In [ ]:
issue['Quantity'].plot(kind='bar', figsize=(14, 6), color='r',rot=0,width=0.95)
plt.title("Parking Citation- Time Near the Hour")
plt.xlabel('Time (24-hour clock )')
plt.ylabel('Number of citations')
plt.show()
issue=issue.sort_values(by = ['Quantity'], ascending = [False])
print (issue.head(issue.shape[0]))

In [ ]:

violation= df.loc[:, ['Violation Description','Fine amount']]
quantity=np.ones(len(violation),dtype=int)
violation['Quantity'] = quantity

violation['Quantity']=violation['Quantity'].groupby(violation['Violation Description']).transform('sum')

violation['%'] = (violation['Quantity']/len(violation))*100

violation=violation.drop_duplicates(['Violation Description']).sort_values(by = ['Quantity'], ascending = [False])

print(violation.head(20))


In [ ]:

@dview.parallel(block = True)
def issue_date(data_frame):
 
  date=numpy.empty(len(data_frame), dtype="S9")
  i=0
  for data in data_frame: 
    temp=datetime.date(int(data[:4]),int(data[5:7]),int(data[8:10]))
    date[i]=calendar.day_name[temp.weekday()]
   
    i+=1
  return date

weekday=pd.DataFrame()
temp= df['Issue Date'].dropna()
weekday['Issue Date']=temp
weekday['Weekday']=issue_date(temp)

quantity=numpy.ones(len(weekday),dtype=int)
weekday['Quantity'] = quantity
weekday['Quantity']=weekday['Quantity'].groupby(weekday['Weekday']).transform('sum')
weekday['%'] = (weekday['Quantity']/len(weekday))*100

weekday=weekday.drop_duplicates(['Weekday']).sort_values(by = ['Quantity'], ascending = [False])
print (weekday.head(weekday.shape[0]))


In [ ]:
total_fine_amount=df['Fine amount'].sum()
print('Total fine amount: $', total_fine_amount)
runtime=time.time()-runtime
print('Runtime:', runtime,'sec')